In [1]:
%cd ..

D:\Course\SBRS


In [128]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm

In [130]:
file_data = 'datasets/onlineretail/online_retail.xlsx'
df = pd.read_csv(file_data)
df.shape

(541909, 3)

In [131]:
df = df[:201]
df.head()

,session_key,item_key,time_key
0,536365,85123A,2010-12-01 08:26:00
1,536365,71053,2010-12-01 08:26:00
2,536365,84406B,2010-12-01 08:26:00
3,536365,84029G,2010-12-01 08:26:00
4,536365,84029E,2010-12-01 08:26:00


In [101]:
# df = df.drop(['Description','Quantity','UnitPrice','userid','Country','datetime'], axis = 1)

In [102]:
df.rename(columns = {'session_key' : 'trans', 'item_key' : 'item'}, inplace = True)

In [103]:
df = df[['trans', 'item']]

In [104]:
df.head()

,trans,item
0,536365,85123A
1,536365,71053
2,536365,84406B
3,536365,84029G
4,536365,84029E


In [105]:
df['item'] = df['item'].astype('str')

In [106]:
unique_items = df.item.unique()
len(unique_items.tolist())


156

In [107]:
df.trans = pd.to_numeric(df.trans, errors='coerce')
df.dropna(axis = 0, how = 'all', inplace = True)
df.trans = df.trans.astype('str')

In [108]:
df = df.groupby('trans').agg(lambda x: x.tolist())

In [109]:
df.head()

,item
trans,
536365.0,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,..."
536366.0,"[22633, 22632]"
536367.0,"[84879, 22745, 22748, 22749, 22310, 84969, 226..."
536368.0,"[22960, 22913, 22912, 22914]"
536369.0,[21756]


In [110]:
class Eclat:
    #инициализация объекта класса
    def __init__(self, min_support = 0.01, max_items = 5, min_items = 2):
        self.min_support = min_support
        self.max_items = max_items
        self.min_items = min_items
        self.item_lst = list()
        self.item_len = 0
        self.item_dict = dict()
        self.final_dict = dict()
        self.data_size = 0
    
    #создание словаря из ненулевых объектов из всех транзакций (вертикальный датасет)
    def read_data(self, dataset):
        for index, row in dataset.iterrows():
            row_wo_na = set(row[0])
            for item in row_wo_na:
                item = item.strip()
                if item in self.item_dict:
                    self.item_dict[item][0] += 1
                else:
                    self.item_dict.setdefault(item, []).append(1)
                self.item_dict[item].append(index)
        #задаем переменные экземпляра (instance variables)
        self.data_size = dataset.shape[0]
        self.item_lst = list(self.item_dict.keys())
        self.item_len = len(self.item_lst)
        self.min_support = self.min_support * self.data_size
        #print ("min_supp", self.min_support)
        
    #рекурсивный метод для поиска всех ItemSet по алгоритму Eclat
    #структура данных: {Item: [Supp number, tid1, tid2, tid3, ...]}
    def recur_eclat(self, item_name, tids_array, minsupp, num_items, k_start):
        if int(tids_array[0]) >= minsupp and num_items <= self.max_items:
            for k in range(k_start+1, self.item_len):
                if self.item_dict[self.item_lst[k]][0] >= minsupp:
                    new_item = item_name + "|" + self.item_lst[k]
                    new_tids = np.intersect1d(tids_array[1:], self.item_dict[self.item_lst[k]][1:])
                    new_tids_size = new_tids.size
                    new_tids = np.insert(new_tids, 0, new_tids_size)
                    if new_tids_size >= minsupp:
                        if num_items >= self.min_items: self.final_dict.update({new_item: new_tids})
                        self.recur_eclat(new_item, new_tids, minsupp, num_items+1, k)
    
    #последовательный вызов функций определенных выше
    def fit(self, dataset):
        i = 0
        self.read_data(dataset)
        for w in tqdm(self.item_lst):
            time.sleep(0.0000001)
            self.recur_eclat(w, self.item_dict[w], self.min_support, 2, i)
            i+=1
        return self
        
    #вывод в форме словаря {ItemSet: support(ItemSet)}
    def transform(self):
        return {k: float("{0:.1f}".format((int(v[0])+0.0)/self.data_size*100)) for k, v in self.final_dict.items()}

In [111]:
model = Eclat(min_support = 0.0001, max_items = 4, min_items = 2)

In [112]:
%%time
model.fit(df)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 156/156 [00:23<00:00,  6.62it/s]

CPU times: total: 21.7 s
Wall time: 23.6 s


In [113]:
rules = model.transform()

In [114]:
rules

{'22752|84029G': 12.5,
 '22752|84029G|84029E': 12.5,
 '22752|84029G|84029E|71053': 12.5,
 '22752|84029G|84029E|21730': 12.5,
 '22752|84029G|84029E|84406B': 12.5,
 '22752|84029G|84029E|85123A': 12.5,
 '22752|84029G|84029E|21071': 8.3,
 '22752|84029G|84029E|82494L': 8.3,
 '22752|84029G|84029E|20679': 8.3,
 '22752|84029G|84029E|37370': 8.3,
 '22752|84029G|84029E|21068': 8.3,
 '22752|84029G|84029E|82486': 8.3,
 '22752|84029G|84029E|21871': 8.3,
 '22752|84029G|84029E|82482': 8.3,
 '22752|84029G|84029E|82483': 8.3,
 '22752|84029G|71053': 12.5,
 '22752|84029G|71053|21730': 12.5,
 '22752|84029G|71053|84406B': 12.5,
 '22752|84029G|71053|85123A': 12.5,
 '22752|84029G|71053|21071': 8.3,
 '22752|84029G|71053|82494L': 8.3,
 '22752|84029G|71053|20679': 8.3,
 '22752|84029G|71053|37370': 8.3,
 '22752|84029G|71053|21068': 8.3,
 '22752|84029G|71053|82486': 8.3,
 '22752|84029G|71053|21871': 8.3,
 '22752|84029G|71053|82482': 8.3,
 '22752|84029G|71053|82483': 8.3,
 '22752|84029G|21730': 12.5,
 '22752|84029

In [115]:
items = df['item']
test = items[13]
test

['22386',
 '85099C',
 '21033',
 '20723',
 '84997B',
 '84997C',
 '21094',
 '20725',
 '21559',
 '22352',
 '21212',
 '21975',
 '21977',
 '84991',
 '84519A',
 '85183B',
 '85071B',
 '21931',
 '21929']

In [ ]:
test_session = ['22752', '21071', '82483', '37370']

'asdasd'

In [117]:
# формируем список по связующим 
def key_check(items_str, count_element):
     return [(key, value) for key, value in rules.items() if key.startswith(items_str) and key.count('|') == count_element]

In [136]:
# находим ранг элемента
def get_rang(check_list, next_elem):
    print(check_list)
    for i, tuple_ in enumerate(check_list):
        if tuple_[0] == next_elem:
#             print(tuple_[0], next_elem)
            return i + 1 
    else:
        return 0 

In [119]:
def str_session(test_session):
    sum_ = 0 
    items_str = test_session[0]
    if len(test_session) == 1: 
        return 0 
    else:
        for i in range(1, len(test_session)): 
            next_elem = items_str + '|' + test_session[i]
            try:
                sum_ += 1 / get_rang(key_check(items_str, i), next_elem)
            except ZeroDivisionError:
                sum_ += 0
            print(items_str, next_elem, sum_)
            items_str, next_elem  = next_elem, items_str

#     return 1 / round(sum_ / len(test_session), 4)
    return sum_

        
str_session(test_session)

22752 22752|21071 0.14285714285714285
22752|21071 22752|21071|82483 0.26785714285714285
22752|21071|82483 22752|21071|82483|37370 0.26785714285714285


0.26785714285714285

In [120]:
test_session = ['22752', '21071', '82483', '37370']

In [142]:
# находим ранг элемента
def get_rank(check_list, next_elem):
    test_dict = {}
    for elem in check_list:
        if elem[1] in test_dict.keys():
            test_dict[elem[1]].append(elem[0])
        else:
            test_dict[elem[1]] = [elem[0]]
    for i, key in enumerate(test_dict.keys()):
        if next_elem in test_dict[key]:
            return i + 1 
    else:
        return 0 

In [144]:
def str_session_v_2(test_session):
    full_test_session_len = len(test_session) - 1
    sum_ = 0
    if len(test_session) == 1: 
        return 0 
    else:
        i = 0
        items_str = test_session[i]
        next_elem = ''
        while i < len(test_session) - 1:
            print('SUM = ',sum_)
            print(items_str, i, len(test_session))
            next_elem = items_str + '|' + test_session[i+1]
            try:
                sum_ += 1 / get_rank(key_check(items_str, i+1), next_elem)
                i += 1
                print('try', items_str, next_elem, sum_)
                items_str, next_elem  = next_elem, ''
            except ZeroDivisionError:
                print('except', items_str, next_elem, sum_) 
                if i > 0:
                    new_items_str = test_session[i]
                    new_next_elem = new_items_str + '|' + test_session[i+1]
                    new_sum = get_rank(key_check(new_items_str, i-1), new_next_elem)
                    if new_sum == 0:
                        items_str = test_session[i+1]
                        test_session = test_session[i+1:]
                        i = 0
                    else:
                        sum_ += 1 / new_sum
                        items_str = new_next_elem 
                    print('except_in', new_items_str, new_next_elem)
                else:
                    items_str = test_session[i+1]
                    test_session = test_session[i+1:]
                    i = 0
       
    return 1/round(sum_ / full_test_session_len,4)

        
str_session_v_2(test_session)

SUM =  0
22752 0 4
try 22752 22752|21071 0.5
SUM =  0.5
22752|21071 1 4
try 22752|21071 22752|21071|82483 1.5
SUM =  1.5
22752|21071|82483 2 4
except 22752|21071|82483 22752|21071|82483|37370 1.5
except_in 82483 82483|37370


0.5

In [133]:
temp = key_check('22752', 1)
temp

[('22752|84029G', 12.5),
 ('22752|84029E', 12.5),
 ('22752|71053', 12.5),
 ('22752|21730', 12.5),
 ('22752|84406B', 12.5),
 ('22752|85123A', 12.5),
 ('22752|21071', 8.3),
 ('22752|82494L', 8.3),
 ('22752|20679', 8.3),
 ('22752|37370', 8.3),
 ('22752|21068', 8.3),
 ('22752|82486', 8.3),
 ('22752|21871', 8.3),
 ('22752|82482', 8.3),
 ('22752|82483', 8.3)]

In [135]:
test_dict = {}
for elem in temp:
    if elem[1] in test_dict.keys():
        test_dict[elem[1]].append(elem[0])
    else:
        test_dict[elem[1]] = [elem[0]]
test_dict

{12.5: ['22752|84029G',
  '22752|84029E',
  '22752|71053',
  '22752|21730',
  '22752|84406B',
  '22752|85123A'],
 8.3: ['22752|21071',
  '22752|82494L',
  '22752|20679',
  '22752|37370',
  '22752|21068',
  '22752|82486',
  '22752|21871',
  '22752|82482',
  '22752|82483']}

In [139]:
for i, key in enumerate(test_dict.keys()):
    if element in test_dict[key]:
        return i + 1

0 12.5
1 8.3


In [ ]:
(0,(12.5, ['22752|84029G',
  '22752|84029E',
  '22752|71053',
  '22752|21730',
  '22752|84406B',
  '22752|85123A']))

In [140]:
# находим ранг элемента
def get_rank(check_list, next_elem):
    test_dict = {}
    for elem in check_list:
        if elem[1] in test_dict.keys():
            test_dict[elem[1]].append(elem[0])
        else:
            test_dict[elem[1]] = [elem[0]]
    for i, key in enumerate(test_dict.keys()):
        if next_elem in test_dict[key]:
            return i + 1 
    else:
        return 0 

In [141]:
next_elem = '22752|21071'
get_rank(key_check('22752', 1), next_elem)

2